# DATA MERGING AND VALIDATION FOR CREDIT RISK ANALYSIS

We will import and join the datasets and explore some missingness of the data

In [187]:
import pandas as pd
import pickle
import numpy as np

In [188]:
trans = pd.read_csv('trans.asc',sep=';')
client = pd.read_csv('client.asc',sep=';')
account = pd.read_csv('account.asc',sep=';')
disp = pd.read_csv('disp.asc',sep=';')
order = pd.read_csv('order.asc',sep=';')
loan = pd.read_csv('loan.asc',sep=';')
card = pd.read_csv('card.asc',sep=';')
district = pd.read_csv('district.asc',sep=';')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Reading and merging the loan and account -related datasets
We merge the static datasets and explore proportional missingness in the full data.

In [189]:
df = pd.merge(loan, account,on='account_id', suffixes=['_loan','_acnt'], how='outer')
df = pd.merge(df, disp, on='account_id', how='outer')
df = pd.merge(df, client, on='client_id', how='outer', suffixes = ['_clnt','_acnt'])
df = pd.merge(df, district, left_on='district_id_clnt', right_on='A1', how='outer')
df = pd.merge(df, card, on='disp_id', how='outer', suffixes=['', '_card'])

In [190]:
print(np.sum(df.isna())/len(df))
print('\nThere are {} loans in the data.'.format(len(df.loan_id.unique())))

loan_id             0.845968
account_id          0.000000
date_loan           0.845968
amount              0.845968
duration            0.845968
payments            0.845968
status              0.845968
district_id_clnt    0.000000
frequency           0.000000
date_acnt           0.000000
disp_id             0.000000
client_id           0.000000
type                0.000000
birth_number        0.000000
district_id_acnt    0.000000
A1                  0.000000
A2                  0.000000
A3                  0.000000
A4                  0.000000
A5                  0.000000
A6                  0.000000
A7                  0.000000
A8                  0.000000
A9                  0.000000
A10                 0.000000
A11                 0.000000
A12                 0.000000
A13                 0.000000
A14                 0.000000
A15                 0.000000
A16                 0.000000
card_id             0.833861
type_card           0.833861
issued              0.833861
dtype: float64

The missingness of credit card data is no issue, as the type and issue date of cards are likely not relevant to the problem. We see that there are relatively few loans available.



### Feature generation for loans

We first drop observations on users that do not have a loan associated in any of the accounts they participate in.

We also drop some irrelevant columns. Most of the identification was only necessary for joining the data, so they are dropped. Most of the demographic data describes essentially the population and urbanization in the area, so this redundant information is dropped.
There are only 5 junior and 3 gold cards in the data so the card type is dropped as well.

We then encode features into formats suitable for machine learning.
WE CREATE

The demographic data is available only as static values measured after some of the loans in the data are already issued. Because ex-ante values are not available, we make the assumption that  the demographics do not drastically change across years. However, if the demographics turn out to be important in predicting credit defaults, this problem should be readdressed.

In [428]:
loans = df[~pd.isna(df.loan_id)]

loans.drop(['account_id','district_id_acnt', 'A1', 'A2', 'A3',
            'A5','A6', 'A7', 'A8', 'A9','district_id_clnt', 'disp_id',
            'client_id', 'card_id', 'type_card'], axis=1, inplace=True)


#create dummy for whether the loan completed successfully
loans['target'] = (loans.status == 'B').astype(int) + (loans.status == 'D').astype(int)

#convert date columns to datetime
loans['date_loan'] = pd.to_datetime(loans.date_loan, format='%y%m%d')
loans['issued'] = pd.to_datetime(loans.issued.str[:6])
loans['date_acnt'] = pd.to_datetime(loans.date_acnt, format='%y%m%d')

#find gender (encoded into the birthnumber) and convert birthdate into datetime
loans['gender'] = (loans.birth_number % 10000 > 5000).astype(int)
loans['birthdate'] = loans.birth_number - 5000 * loans.gender + 19000000
loans['birthdate'] = pd.to_datetime(loans.birthdate, format='%Y%m%d')

#find the age of applicant and the account at the time of loan issuance
loans['appl_age'] = (loans.date_loan - loans.birthdate).dt.days / 365.25
loans['accnt_age'] = (loans.date_loan - loans.date_acnt).dt.days / 365.25



#create dummy for whether the account has an associated card at the time of loan issuance
loans['issued'] = (loans.issued < loans.date_loan).astype(int)

#create dummies for the frequency of statement issuance and the account type
loans = pd.get_dummies(loans, columns=['frequency', 'type'], drop_first=True)


# select unemployment and crime from the demographic statistics.
loans['unempl'] = np.select([loans.date_loan.dt.year > 1996,
                             loans.date_loan.dt.year < 1997],
                            [loans.A13, loans.A12])

loans['crime'] = np.select([loans.date_loan.dt.year > 1996, loans.date_loan.dt.year < 1997],
          [loans.A16, loans.A15])

# convert the columns to numeric values and scale the crime numbers for population
loans['unempl'] = pd.to_numeric(loans.unempl, errors='coerce')
loans['crime'] = pd.to_numeric(loans.crime, errors='coerce') / loans.A4
loans['A14'] = loans.A14 / loans.A4 * 100

#finally, aggregate to loan-level from client-level data
loans = loans.groupby('loan_id').agg('mean')
loans['multi'] = np.select([loans.type_OWNER < 1], [1], 0)

#drop unnecessary columns
loans.drop(['birth_number','type_OWNER','A13','A16'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [430]:
loans.columns = ['amount', 'duration', 'payments', 'pop', 'urban_rat', 'avg_sal', 'rat_urban', 'card',
       'target', 'gender', 'appl_age', 'accnt_age',
       'freq_trans', 'freq_weekly', 'unempl',
       'crime', 'multi']

In [431]:
print(np.sum(loans.isna()))
print('\nThere are {} loans in the data.'.format(len(loans)))

amount         0
duration       0
payments       0
pop            0
urban_rat      0
avg_sal        0
rat_urban      0
card           0
target         0
gender         0
appl_age       0
accnt_age      0
freq_trans     0
freq_weekly    0
unempl         3
crime          3
multi          0
dtype: int64

There are 682 loans in the data.


### Read in and merge transaction data

Note that, because the dataset is relatively small, we introduce some redundancy for a while by joining the full dataset of static information. 
This is done to spare lines of code.

In [191]:
trans = pd.read_csv('trans.asc', sep=';')

In [192]:
trans = pd.merge(trans, df, on='account_id', suffixes=['_trans', ''], how='left')
#Subset the data set to transactions for accounts with loans:
trans_loans = trans[~pd.isna(trans.loan_id)]

In [193]:
print('Total transactions: ', len(trans))
print('Transactions for accounts associated with a loan: ', len(trans_loans))

Total transactions:  1262625
Transactions for accounts associated with a loan:  233627


### Datetime wrangling

Because we're interested in predicting bad loans, we should use transaction data from only prior to giving out the loan.
In order to work with the dates, we will first transform them to datetime format

In [194]:
trans_loans.date = pd.to_datetime(trans_loans.date, format='%y%m%d')
trans_loans.date_loan = pd.to_datetime(trans_loans.date_loan, format='%y%m%d')

#filter to transactions prior to loan issuance
trans_loans = trans_loans[trans_loans.date < trans_loans.date_loan]

print('We end up with {} ex-ante transactions for our final data.'.format(len(trans_loans)))

We end up with 66762 ex-ante transactions for our final data.


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### Explore missingness in the transaction data 

In [195]:
print(np.sum(trans.isna())/len(trans))

trans_id            0.000000
account_id          0.000000
date                0.000000
type_trans          0.000000
operation           0.172902
amount_trans        0.000000
balance             0.000000
k_symbol            0.451296
bank                0.733849
account             0.713619
loan_id             0.814967
date_loan           0.814967
amount              0.814967
duration            0.814967
payments            0.814967
status              0.814967
district_id_clnt    0.000000
frequency           0.000000
date_acnt           0.000000
disp_id             0.000000
client_id           0.000000
type                0.000000
birth_number        0.000000
district_id_acnt    0.000000
A1                  0.000000
A2                  0.000000
A3                  0.000000
A4                  0.000000
A5                  0.000000
A6                  0.000000
A7                  0.000000
A8                  0.000000
A9                  0.000000
A10                 0.000000
A11           

In [196]:
print(np.sum(trans_loans.isna())/len(trans_loans))

trans_id            0.000000
account_id          0.000000
date                0.000000
type_trans          0.000000
operation           0.172883
amount_trans        0.000000
balance             0.000000
k_symbol            0.582322
bank                0.815763
account             0.706180
loan_id             0.000000
date_loan           0.000000
amount              0.000000
duration            0.000000
payments            0.000000
status              0.000000
district_id_clnt    0.000000
frequency           0.000000
date_acnt           0.000000
disp_id             0.000000
client_id           0.000000
type                0.000000
birth_number        0.000000
district_id_acnt    0.000000
A1                  0.000000
A2                  0.000000
A3                  0.000000
A4                  0.000000
A5                  0.000000
A6                  0.000000
A7                  0.000000
A8                  0.000000
A9                  0.000000
A10                 0.000000
A11           

Bank and account columns report the bank and account of a partner in a transaction. As such, the specific bank and of a partner are likely not relevant and missingness should not be an issue.

Because we want to utilize the transactions for predicting credit defaults, transaction types are intuitively important. The operation and k_symbol columns describe the transaction type. We will explore the missing operation data further by examining, whethere there are cases where both operation and k_symbol are missing:

In [197]:
print('Percentage of cases where both operation and k_symbol are missing: ')
print(np.sum(trans.operation.isna() & trans.k_symbol.isna())/len(trans))
print('Percentage of cases where both operation and k_symbol are reported: ')
print(np.sum(~trans.operation.isna() & ~trans.k_symbol.isna())/len(trans))

print('Percentage of cases where both operation and k_symbol are missing: ')
print(np.sum(trans_loans.operation.isna() & trans_loans.k_symbol.isna())/len(trans))
print('Percentage of cases where both operation and k_symbol are reported: ')
print(np.sum(~trans_loans.operation.isna() & ~trans_loans.k_symbol.isna())/len(trans))

Percentage of cases where both operation and k_symbol are missing: 
0.0
Percentage of cases where both operation and k_symbol are reported: 
0.3758019998019998
Percentage of cases where both operation and k_symbol are missing: 
0.0
Percentage of cases where both operation and k_symbol are reported: 
0.012943668943668944


We see that, while operation and k_symbol are missing at times, there are no cases where neither one is present. Thus, missingness should be no issue.

Because missing data is not an issue and we do not need to use the transaction data to predict missing values, we can focus on the subset of the transactions data, where there are loans present.



## Aggregating transactions

For now, we'll drop the other info and focus on aggregating the transactions data to loan-level.

In [198]:
trans_loans = trans_loans[['loan_id', 'date','date_loan', 'type_trans', 'operation',
       'amount_trans', 'balance', 'k_symbol', 'bank', 'account']]


trans_loans = pd.get_dummies(trans_loans, columns=['type_trans', 'operation','k_symbol'])
trans_loans['bank'] = trans_loans.bank.isna().astype('int')
trans_loans['account'] = trans_loans.account.isna().astype('int')


In [199]:

trans_agg = trans_loans.groupby('loan_id').agg('mean')

In [202]:
trans_agg = trans_agg.join(trans_loans.loan_id.value_counts(sort=False))

In [178]:

loans = pd.merge(loans, trans_agg, left_on='loan_id', right_index=True, suffixes=['','_trans'], how='left')
loans

,loan_id,account_id,date_loan,amount,duration,payments,status,district_id_clnt,frequency,date_acnt,...,operation_VKLAD,operation_VYBER,operation_VYBER KARTOU,k_symbol_,k_symbol_POJISTNE,k_symbol_SANKC. UROK,k_symbol_SIPO,k_symbol_SLUZBY,k_symbol_UROK,transactions
0,5314.0,1787,930705.0,96396.0,12.0,8033.0,B,30,POPLATEK TYDNE,930322,...,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,6903.0,9337,940604.0,52128.0,24.0,2172.0,A,30,POPLATEK MESICNE,930913,...,0.183099,0.450704,0.0,0.000000,0.000000,0.084507,0.000000,0.070423,0.366197,1
44,5316.0,1801,930711.0,165960.0,36.0,4610.0,A,46,POPLATEK MESICNE,930213,...,0.324324,0.324324,0.0,0.081081,0.081081,0.000000,0.054054,0.081081,0.135135,1
45,6554.0,7637,970129.0,390096.0,48.0,8127.0,C,46,POPLATEK MESICNE,960819,...,0.375000,0.375000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,1
46,6554.0,7637,970129.0,390096.0,48.0,8127.0,C,46,POPLATEK MESICNE,960819,...,0.375000,0.375000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5251,7171.0,10650,980224.0,196224.0,48.0,4088.0,C,76,POPLATEK MESICNE,970909,...,0.043478,0.434783,0.0,0.000000,0.000000,0.000000,0.043478,0.043478,0.217391,1
5252,7200.0,10799,980711.0,31140.0,60.0,519.0,C,76,POPLATEK MESICNE,970819,...,0.340000,0.460000,0.0,0.000000,0.000000,0.000000,0.000000,0.120000,0.200000,1
5253,6969.0,9673,980912.0,88200.0,36.0,2450.0,C,76,POPLATEK MESICNE,970607,...,0.054945,0.472527,0.0,0.000000,0.000000,0.000000,0.131868,0.120879,0.164835,1
5254,6969.0,9673,980912.0,88200.0,36.0,2450.0,C,76,POPLATEK MESICNE,970607,...,0.054945,0.472527,0.0,0.000000,0.000000,0.000000,0.131868,0.120879,0.164835,1


In [203]:
trans_agg

,amount_trans,balance,bank,account,type_trans_PRIJEM,type_trans_VYBER,type_trans_VYDAJ,operation_PREVOD NA UCET,operation_PREVOD Z UCTU,operation_VKLAD,operation_VYBER,operation_VYBER KARTOU,k_symbol_,k_symbol_POJISTNE,k_symbol_SANKC. UROK,k_symbol_SIPO,k_symbol_SLUZBY,k_symbol_UROK,loan_id
loan_id,,,,,,,,,,,,,,,,,,,
4959.0,7954.257407,32590.624074,0.722222,0.611111,0.407407,0.055556,0.537037,0.092593,0.185185,0.037037,0.500000,0.000000,0.000000,0.000000,0.000000,0.092593,0.111111,0.185185,108
4961.0,5856.300000,25197.092500,0.837500,0.625000,0.575000,0.025000,0.400000,0.000000,0.162500,0.112500,0.425000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.300000,80
4962.0,12113.931098,62991.264634,0.737805,0.652439,0.310976,0.036585,0.652439,0.262195,0.000000,0.219512,0.426829,0.000000,0.091463,0.085366,0.000000,0.085366,0.085366,0.091463,164
4967.0,7571.999138,39953.970690,0.689655,0.586207,0.275862,0.017241,0.706897,0.310345,0.000000,0.155172,0.413793,0.000000,0.103448,0.103448,0.008621,0.103448,0.094828,0.120690,116
4968.0,4716.194545,31383.449091,0.600000,0.509091,0.418182,0.018182,0.563636,0.236364,0.163636,0.109091,0.345455,0.000000,0.090909,0.072727,0.000000,0.072727,0.072727,0.145455,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7294.0,7977.972222,55438.851852,0.888889,0.777778,0.425926,0.000000,0.574074,0.111111,0.000000,0.240741,0.462963,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.185185,54
7295.0,8138.691509,38619.102830,1.000000,0.858491,0.443396,0.047170,0.509434,0.000000,0.000000,0.264151,0.556604,0.000000,0.000000,0.000000,0.000000,0.000000,0.141509,0.179245,106
7304.0,24426.472222,59352.833333,0.666667,0.611111,0.611111,0.000000,0.388889,0.055556,0.277778,0.111111,0.333333,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.222222,36


## Export final transactions data



In [ ]:
with open('transactions_data','wb') as file:
    pickle.dump(trans_loans, file)

## Some additional data wrangling

,trans_id,account_id,date,amount_trans,balance,account,loan_id,date_loan,amount,duration,...,A6,A7,A8,A9,A10,A11,A13,A14,A16,card_id
count,2.336270e+05,233627.000000,233627.000000,233627.000000,233627.000000,7.707300e+04,233627.000000,233627.000000,233627.000000,233627.000000,...,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,47127.000000
mean,2.084692e+06,6034.333245,966293.534476,8239.434745,45520.418867,4.067135e+07,6216.416921,956830.527619,147130.014733,35.513121,...,21.533367,5.469137,1.682785,5.291794,68.555529,9551.048946,3.468643,122.373737,17582.671104,820.616738
std,1.115108e+06,3233.873384,13622.799197,11740.717669,24765.050048,3.272112e+07,672.007484,14668.160340,111462.998365,17.255362,...,16.112486,4.601195,1.071525,2.907060,20.480304,1356.102185,2.142688,23.791757,32637.127783,327.342498
min,2.760000e+02,2.000000,930113.000000,0.000000,-19310.000000,0.000000e+00,4959.000000,930705.000000,4980.000000,12.000000,...,0.000000,0.000000,0.000000,1.000000,33.900000,8110.000000,0.430000,81.000000,888.000000,16.000000
25%,1.113868e+06,3115.000000,960307.000000,199.500000,27441.450000,7.603472e+06,5611.000000,941003.000000,67464.000000,24.000000,...,8.000000,2.000000,1.000000,4.000000,52.700000,8546.000000,1.960000,106.000000,2252.000000,587.000000
50%,2.279742e+06,6148.000000,970508.000000,3600.000000,40934.000000,3.866864e+07,6253.000000,960119.000000,108144.000000,36.000000,...,23.000000,5.000000,1.000000,6.000000,61.900000,8994.000000,3.490000,117.000000,3868.000000,890.000000
75%,3.091876e+06,8784.000000,980305.000000,10530.000000,59333.200000,6.941577e+07,6766.000000,970518.000000,203940.000000,48.000000,...,33.000000,8.000000,2.000000,7.000000,87.700000,9897.000000,4.720000,137.000000,6872.000000,1081.000000
max,3.682987e+06,11362.000000,981231.000000,87300.000000,209637.000000,9.999420e+07,7308.000000,981208.000000,590820.000000,60.000000,...,70.000000,20.000000,5.000000,11.000000,100.000000,12541.000000,9.400000,167.000000,99107.000000,1247.000000
